In [1]:
#Importar bibliotecas do sistemas
import sys
import os

# Caminho absoluto até a pasta src
sys.path.append(os.path.abspath("../src"))

#Importar libs do python
import pandas as pd 
import numpy as np 
import math
from pyautocad import Autocad, APoint 

#Importar libs do programa 
import design_functions
from v_p_chapa_extremidade.v_p_chapa_extremidade import *
from draw_autocad.draw_autocad_figures import *
from materials import * 

In [2]:
perfil_escolhido = W_530x101_0


aço_escolhido = ASTM_A36

solda_escolhida =E70XX

perfil_escolhido.inercias()
perfil_escolhido.material(aço_escolhido)


In [3]:
M = 0 #kN.mm 

V = 499 #kN

T = 0 #kN

parafuso = ASTM_A325
parafuso.prop_geometricas(rosca = 1,planos_de_corte =1)

In [4]:
rigida =1

filete_duplo =2 

[chapa,exp,parafuso,ver_parafuso,solda,esp_solda] = dim_chapa_parafuso(V,T,perfil_escolhido,parafuso,aço_escolhido,rigida,solda_escolhida,filete_duplo,gamma)

In [5]:
chapa.df

,vértice,x (mm),y (mm)
0,1,0,0.0
1,2,200,0.0
2,3,200,502.2
3,4,0,502.2
4,5,0,0.0


In [6]:
perfil_escolhido.h_w

482.2

In [7]:
exp

15.875

In [8]:
parafuso.diametro_mm

19.049999999999997

In [9]:
ver_parafuso

,parafuso,x (mm),y (mm)
0,1,40,25
1,2,160,25
2,3,40,100
3,4,160,100
4,5,40,175
5,6,160,175
6,7,40,250
7,8,160,250
8,9,40,325
9,10,160,325


##### Abertura do projeto no pyautocad

In [10]:
acad = Autocad(create_if_not_exists=True)
acad.prompt("Hello, Autocad from Python\n")
print(acad.doc.Name)

Hello, Autocad from Python

Desenho1.dwg


In [11]:
limpar_desenho(acad)

✅ Desenho limpo com sucesso.


In [12]:
pontos_hexagono = gerar_pontos_hexagono(parafuso.diametro_mm)

In [13]:
# Chamando a função para desenhar a chapa 3D
objetos_chapa = criar_chapa_3d(acad, chapa.df, exp)

In [14]:
py = (perfil_escolhido.t_f)
objetos_secao_perfil = desenhar_secao_perfil(acad, perfil_escolhido, (chapa.B / 2) - (perfil_escolhido.b_f / 2), posicao_y=(-py), altura_z=exp)

In [15]:
objetos_parafusos=[]
for i in range(ver_parafuso.shape[0]):
    x_centro = ver_parafuso.iat[i, 1]
    y_centro = ver_parafuso.iat[i, 2]

    # Adicionar circunferência no ponto
    obj = acad.model.AddCircle(APoint(x_centro, y_centro,exp), parafuso.diametro_mm / 2)
    objetos_parafusos.append(obj)
    obj = acad.model.AddCircle(APoint(x_centro, y_centro,0), parafuso.diametro_mm / 2)
    objetos_parafusos.append(obj)
    # Transladar hexágono para o ponto atual
    hexagono_transladado = transladar_pontos(pontos_hexagono, x_centro, y_centro, exp)

    for j in range(len(hexagono_transladado) - 1):
        p1 = APoint(*hexagono_transladado[j])
        p2 = APoint(*hexagono_transladado[j + 1])
        obj = acad.model.AddLine(p1, p2)
        objetos_parafusos.append(obj)

In [16]:
# Rotacionar apenas a seção do perfil:
for obj in objetos_parafusos:
    obj.Rotate3D(APoint(0, 0, 0), APoint(1,0, 0), math.radians(90))
    obj.Rotate3D(APoint(0, 0, 0), APoint(0,0, 1), math.radians(90))

for obj in objetos_chapa:
    obj.Rotate3D(APoint(0, 0, 0), APoint(1,0, 0), math.radians(90))
    obj.Rotate3D(APoint(0, 0, 0), APoint(0,0, 1), math.radians(90))

for obj in objetos_secao_perfil:
    obj.Rotate3D(APoint(0, 0, 0), APoint(1,0, 0), math.radians(90))
    obj.Rotate3D(APoint(0, 0, 0), APoint(0,0, 1), math.radians(90))

# Vetor de translação (exemplo: mover 100 mm no eixo X)
dx, dy, dz = 0,-perfil_escolhido.b_f/2,0  # ajuste aqui conforme necessário

# Aponta o vetor de deslocamento
vetor = APoint(dx, dy, dz)

for obj in objetos_secao_perfil:
    obj.Move(APoint(0,0,0),vetor)


for obj in objetos_chapa:
    obj.Move(APoint(0,0,0),vetor)


for obj in objetos_parafusos:
    obj.Move(APoint(0,0,0),vetor)

In [17]:
obs="(A solda será colocada em todo contorno da viga, com a chapa, inclusive na mesa da viga, sendo uma solda com face superior da chapa)"
escrever_descricao(acad,-perfil_escolhido.b_f*0.75,0,perfil_escolhido.h-10 ,"Chapa","",perfil_escolhido.nome,math.ceil(esp_solda),obs)